In [1]:
import os
import requests
from bs4 import BeautifulSoup as bs

In [ ]:
aadd

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta, date
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException
import time 
import pandas as pd
from io import StringIO
import re

In [7]:
def move_to_next_page(driver):
    try:
        # 현재 페이지를 나타내는 strong 요소 찾기
        current_page = driver.find_element(By.CSS_SELECTOR, "#paging strong")
        # 다음 a 태그 찾기 (current_page 다음에 나오는 a)
        next_page = current_page.find_element(By.XPATH, "following-sibling::a[1]")
        
        # 다음 페이지가 있으면 클릭
        driver.execute_script("arguments[0].click();", next_page)
        time.sleep(3)  # 페이지가 넘어갈 시간 대기
        print("✅ 다음 페이지로 이동합니다")
        return True
    except Exception as e:
        print("✅ 다음 페이지 없음, 크롤링 종료")
        return False

In [5]:
keyword1 = input("지역을 입력해주세요")
keyword2 = input("주거용과 아파트 중 하나를 입력해주세요")

지역을 입력해주세요경기
주거용과 아파트 중 하나를 입력해주세요아파트


In [8]:

options = Options()
options.add_experimental_option("detach", True)
options.add_argument("start-maximized")
options.add_argument("Chrome/135.0.0.0")
options.add_argument("lang=ko_KR")


# --- 드라이버 실행 ---
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 10)

# --- 로그인 ---
driver.get('https://www.tankauction.com/')
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.hand'))).click()
wait.until(EC.element_to_be_clickable((By.ID, 'client_id'))).send_keys('yjj7479')
driver.find_element(By.ID, 'passwd').send_keys('xodzmdhrtus0911!')
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, ".button.btn_white.bigrounded").click()
time.sleep(3)

# --- 경매 검색 ---
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".menuicon_10"))).click()
Select(wait.until(EC.element_to_be_clickable((By.ID, "siCd")))).select_by_visible_text(keyword1)
time.sleep(1)
Select(wait.until(EC.element_to_be_clickable((By.ID, "ctgr")))).select_by_visible_text(keyword2)
time.sleep(1)
Select(wait.until(EC.element_to_be_clickable((By.ID, "minbAmtEnd")))).select_by_visible_text('4억')

# --- 날짜 설정 ---
today = date.today()
after_month = today + timedelta(days=30)
start_str = today.strftime("%Y-%m-%d")
end_str = after_month.strftime("%Y-%m-%d")

wait.until(EC.element_to_be_clickable((By.ID, "bgnDt"))).send_keys(start_str)
wait.until(EC.element_to_be_clickable((By.ID, "endDt"))).send_keys(end_str)

# --- 검색 및 리스트 로딩 ---
wait.until(EC.element_to_be_clickable((By.ID, "btnSrch"))).click()
Select(wait.until(EC.element_to_be_clickable((By.ID, "dataSize_s")))).select_by_visible_text('100')
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#lsTbody > tr.hand')))
WebDriverWait(driver, 10).until(lambda d: d.execute_script("return document.readyState") == "complete")

print("\n🔍 리스트 로딩 완료")

# --- 결과 저장용 ---
result1 = []
cols = ['입찰여부','사건번호', '주소','면적','사용승인연도','감정가','최저가','최대입찰가','입찰가','비용','실거래가','예상 낙찰비율','낙찰가','오늘조회수','누적조회수','입찰일']

# --- 페이지별 데이터 수집 ---
while True:
    # 스크롤 끝까지 내려서 전체 tr 다 로딩되게 만들기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # 새로 로딩된 tr 요소들 다시 수집
    trs = driver.find_elements(By.CSS_SELECTOR, "#lsTbody > tr.hand")
    print(f"🔵 현재 페이지 tr 개수: {len(trs)}개")

    # tr 하나하나 수집
    for idx, tr in enumerate(trs, start=1):
        try:
            tid = tr.find_element(By.CSS_SELECTOR, 'input[id^="Tid_"]').get_attribute("value")
            num = tr.find_element(By.CSS_SELECTOR, ".clear.f15.bold").text
            address = tr.find_element(By.CSS_SELECTOR, f'div#adrs_{tid}').get_attribute("innerText")
            start_price = int(tr.find_element(By.ID, f"apslAmt_{tid}").text.replace(",", ""))
            end_price = int(tr.find_element(By.ID, f"minbAmt_{tid}").text.replace(",", ""))
            the_date = tr.find_element(By.ID, f"bidDt_{tid}").text

            # 팝업 열기
            main_window = driver.current_window_handle
            clickable = tr.find_elements(By.CSS_SELECTOR, 'td[onclick^="cntsViewPN"]')[1]
            driver.execute_script("arguments[0].scrollIntoView(true);", clickable)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", clickable)

            # 팝업 전환
            WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
            new_window = [w for w in driver.window_handles if w != main_window][0]
            driver.switch_to.window(new_window)

            # 팝업 데이터 수집
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#lyCnt_object")))
            area = driver.find_element(By.CSS_SELECTOR, '#lyCnt_object > table.Ltbl_list > tbody:nth-child(4) > tr > td:nth-child(3)').text

            hit_info = driver.find_element(By.CSS_SELECTOR,'#hitInfo > span').text
            hit_info2 = driver.find_element(By.CSS_SELECTOR,'#hitInfo > span:nth-child(2)').text
            
            try:
                # "매각사례" 섹션 찾기
                auction_sections = driver.find_elements(By.CSS_SELECTOR, "#lyCnt_add .sectTitle")
                auction_table = None

                for section in auction_sections:
                    title_element = section.find_element(By.CSS_SELECTOR, "dt")
                    title_text = title_element.text.strip()

                    if "매각사례" in title_text:
                        # 찾으면 바로 다음 테이블
                        auction_table = section.find_element(By.XPATH, "following-sibling::table[1]")
                        break

                if auction_table is None:
                    raise Exception("매각사례 테이블을 찾지 못했습니다.")

                rows = auction_table.find_elements(By.TAG_NAME, "tr")

                if len(rows) > 1:
                    first_row = rows[1]
                    cells = first_row.find_elements(By.TAG_NAME, "td")
                    
                    if len(cells) >= 4:
                        raw_text = cells[3].text.strip()
                        # 정규식으로 "숫자.숫자%" 만 필터
                        import re
                        match = re.search(r"\d+\.\d+%", raw_text)
                        if match:
                            average_sale_rate = match.group()
                        else:
                            average_sale_rate = None
                    else:
                        average_sale_rate = None
                else:
                    average_sale_rate = None

                print(f"✅ 예상 낙찰 비율 추출 성공: {average_sale_rate}")

            except Exception as e:
                print(f"⚠️ 예상 낙찰 비율 추출 실패: {e}")
                average_sale_rate = None


            # 실거래가
            try:
                real_price_html = driver.find_element(By.CSS_SELECTOR, "#qtLst > tbody > tr > td > div:nth-child(1)").get_attribute("outerHTML")
                df_real = pd.read_html(StringIO(real_price_html))[0]
                real_price = round(df_real['평균가'].apply(lambda x: int(str(x).replace(",", ""))).mean()*10000)
            except:
                real_price = 0

            # 인테리어 비용
            approval_elements = driver.find_elements(By.CSS_SELECTOR, "span.show_b")
            year = 9999
            for elem in approval_elements:
                text = elem.text.strip()
                if "사용승인일" in text:
                    year = int(text.split(":")[1].split("-")[0])
                    break

            if year < 2010:
                try:
                    size_text = driver.find_element(By.CSS_SELECTOR, "#Tr_4 > td:nth-child(3) > div:nth-child(6) > span").text
                    area_size = int(size_text.split("(")[1].split("평")[0])
                    interior = area_size * 1000000
                except:
                    interior = 3000000
            else:
                interior = 3000000

            expense = (start_price * 0.011) + (start_price * 0.004) + 2000000 + (40000000 * 0.24) + interior + (end_price * 0.7 * 0.05 / 2) + 600000 + 1000000
            max_pice = end_price + expense + 10000000
            decision = 1 if max_pice < real_price else 0

            # 팝업 닫고 메인으로 복귀
            driver.close()
            driver.switch_to.window(main_window)

            # 결과 저장
            result1.append([decision, num, address, area, year, start_price, end_price, max_pice, None, expense, real_price, average_sale_rate, None, hit_info, hit_info2, the_date])
            print(f"✅ {idx}번째 tr 추출 완료")

            time.sleep(2)

        except Exception as e:
            print(f"⚠️ {idx}번째 tr 추출 실패: {e}")

    # 다음 페이지 없으면 break
    if not move_to_next_page(driver):
        print("✅ 다음 페이지 없음, 크롤링 종료")
        break

# --- 결과 저장 ---
df = pd.DataFrame(result1, columns=cols)
df.to_csv(f"./[{today.year}년{today.month:02d}월]{keyword1}_{keyword2}_경매결과.csv", encoding="utf-8-sig", index=False)
print("✅ 파일 저장 완료")




🔍 리스트 로딩 완료
🔵 현재 페이지 tr 개수: 100개
✅ 예상 낙찰 비율 추출 성공: 89.15%
✅ 1번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 97.99%
✅ 2번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 83.66%
✅ 3번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 83.66%
✅ 4번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 90.75%
✅ 5번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: None
✅ 6번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 90.75%
✅ 7번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 94.46%
✅ 8번째 tr 추출 완료
⚠️ 예상 낙찰 비율 추출 실패: 매각사례 테이블을 찾지 못했습니다.
✅ 9번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 96.83%
✅ 10번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 81.15%
✅ 11번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 91.06%
✅ 12번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 97.01%
✅ 13번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 69.93%
✅ 14번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 84.47%
✅ 15번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 84.34%
✅ 16번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 84.87%
✅ 17번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 97.38%
✅ 18번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 83.24%
✅ 19번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 87.21%
✅ 20번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 84.87%
✅ 21번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 69.93%
✅ 22번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 69.93%
✅ 23번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 84

✅ 예상 낙찰 비율 추출 성공: 88.44%
✅ 98번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 88.44%
✅ 99번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 101.14%
✅ 100번째 tr 추출 완료
✅ 다음 페이지로 이동합니다
🔵 현재 페이지 tr 개수: 100개
✅ 예상 낙찰 비율 추출 성공: 87.24%
✅ 1번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 96.87%
✅ 2번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 84.05%
✅ 3번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 84.05%
✅ 4번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 93.06%
✅ 5번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 90.69%
✅ 6번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 90.11%
✅ 7번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 85.4%
✅ 8번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 96.28%
✅ 9번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 85.53%
✅ 10번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 85.99%
✅ 11번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 93.27%
✅ 12번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 97.45%
✅ 13번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 85.86%
✅ 14번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 90.79%
✅ 15번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 86.52%
✅ 16번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 74.51%
✅ 17번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 80.67%
✅ 18번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 95.25%
✅ 19번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 87.8%
✅ 20번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 87.8%
✅ 21번

In [ ]:
df.to_csv(f"/Users/jihyun/Desktop/자료/경매/입찰결정/[{today.year}년{today.month:02d}월]{keyword1}_{keyword2}_경매결과.csv", encoding="utf-8-sig", index=False)
print("✅ 파일 저장 완료")

In [13]:
keyword1 = input("지역을 입력해주세요")
keyword2 = input("주거용과 아파트 중 하나를 입력해주세요")

지역을 입력해주세요서울
주거용과 아파트 중 하나를 입력해주세요주거용


In [27]:

options = Options()
options.add_experimental_option("detach", True)
options.add_argument("start-maximized")
options.add_argument("Chrome/135.0.0.0")
options.add_argument("lang=ko_KR")


# --- 드라이버 실행 ---
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 10)

# --- 로그인 ---
driver.get('https://www.tankauction.com/')
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.hand'))).click()
wait.until(EC.element_to_be_clickable((By.ID, 'client_id'))).send_keys('yjj7479')
driver.find_element(By.ID, 'passwd').send_keys('xodzmdhrtus0911!')
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, ".button.btn_white.bigrounded").click()
time.sleep(3)

# --- 경매 검색 ---
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".menuicon_10"))).click()
Select(wait.until(EC.element_to_be_clickable((By.ID, "siCd")))).select_by_visible_text(keyword1)
time.sleep(1)
Select(wait.until(EC.element_to_be_clickable((By.ID, "ctgr")))).select_by_visible_text(keyword2)
time.sleep(1)
Select(wait.until(EC.element_to_be_clickable((By.ID, "minbAmtEnd")))).select_by_visible_text('4억')

# --- 날짜 설정 ---
today = date.today()
after_month = today + timedelta(days=30)
start_str = today.strftime("%Y-%m-%d")
end_str = after_month.strftime("%Y-%m-%d")

wait.until(EC.element_to_be_clickable((By.ID, "bgnDt"))).send_keys(start_str)
wait.until(EC.element_to_be_clickable((By.ID, "endDt"))).send_keys(end_str)

# --- 검색 및 리스트 로딩 ---
wait.until(EC.element_to_be_clickable((By.ID, "btnSrch"))).click()
Select(wait.until(EC.element_to_be_clickable((By.ID, "dataSize_s")))).select_by_visible_text('100')
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#lsTbody > tr.hand')))
WebDriverWait(driver, 10).until(lambda d: d.execute_script("return document.readyState") == "complete")

print("\n🔍 리스트 로딩 완료")

# --- 결과 저장용 ---
result1 = []
cols = ['입찰여부','사건번호', '주소','면적','사용승인연도','주차수','감정가','최저가','최대입찰가','입찰가','비용','실거래가','예상 낙찰비율','낙찰가','오늘조회수','누적조회수','입찰일']

# --- 페이지별 데이터 수집 ---
while True:
    # 스크롤 끝까지 내려서 전체 tr 다 로딩되게 만들기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # 새로 로딩된 tr 요소들 다시 수집
    trs = driver.find_elements(By.CSS_SELECTOR, "#lsTbody > tr.hand")
    print(f"🔵 현재 페이지 tr 개수: {len(trs)}개")

    # tr 하나하나 수집
    for idx, tr in enumerate(trs, start=1):
        try:
            tid = tr.find_element(By.CSS_SELECTOR, 'input[id^="Tid_"]').get_attribute("value")
            num = tr.find_element(By.CSS_SELECTOR, ".clear.f15.bold").text
            address = tr.find_element(By.CSS_SELECTOR, f'div#adrs_{tid}').get_attribute("innerText")
            start_price = int(tr.find_element(By.ID, f"apslAmt_{tid}").text.replace(",", ""))
            end_price = int(tr.find_element(By.ID, f"minbAmt_{tid}").text.replace(",", ""))
            the_date = tr.find_element(By.ID, f"bidDt_{tid}").text

            # 팝업 열기
            main_window = driver.current_window_handle
            clickable = tr.find_elements(By.CSS_SELECTOR, 'td[onclick^="cntsViewPN"]')[1]
            driver.execute_script("arguments[0].scrollIntoView(true);", clickable)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", clickable)

            # 팝업 전환
            WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
            new_window = [w for w in driver.window_handles if w != main_window][0]
            driver.switch_to.window(new_window)

            # 팝업 데이터 수집
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#lyCnt_object")))
            area = driver.find_element(By.CSS_SELECTOR, '#lyCnt_object > table.Ltbl_list > tbody:nth-child(4) > tr > td:nth-child(3)').text

            hit_info = driver.find_element(By.CSS_SELECTOR,'#hitInfo > span').text
            hit_info2 = driver.find_element(By.CSS_SELECTOR,'#hitInfo > span:nth-child(2)').text
            
            try:
                # "매각사례" 섹션 찾기
                auction_sections = driver.find_elements(By.CSS_SELECTOR, "#lyCnt_add .sectTitle")
                auction_table = None

                for section in auction_sections:
                    title_element = section.find_element(By.CSS_SELECTOR, "dt")
                    title_text = title_element.text.strip()

                    if "매각사례" in title_text:
                        # 찾으면 바로 다음 테이블
                        auction_table = section.find_element(By.XPATH, "following-sibling::table[1]")
                        break

                if auction_table is None:
                    raise Exception("매각사례 테이블을 찾지 못했습니다.")

                rows = auction_table.find_elements(By.TAG_NAME, "tr")

                if len(rows) > 1:
                    first_row = rows[1]
                    cells = first_row.find_elements(By.TAG_NAME, "td")
                    
                    if len(cells) >= 4:
                        raw_text = cells[3].text.strip()
                        # 정규식으로 "숫자.숫자%" 만 필터
                        import re
                        match = re.search(r"\d+\.\d+%", raw_text)
                        if match:
                            average_sale_rate = match.group()
                        else:
                            average_sale_rate = None
                    else:
                        average_sale_rate = None
                else:
                    average_sale_rate = None

                print(f"✅ 예상 낙찰 비율 추출 성공: {average_sale_rate}")

            except Exception as e:
                print(f"⚠️ 예상 낙찰 비율 추출 실패: {e}")
                average_sale_rate = None


            # 실거래가
            try:
                real_price_html = driver.find_element(By.CSS_SELECTOR, "#qtLst > tbody > tr > td > div:nth-child(1)").get_attribute("outerHTML")
                df_real = pd.read_html(StringIO(real_price_html))[0]
                real_price = round(df_real['평균가'].apply(lambda x: int(str(x).replace(",", ""))).mean()*10000)
            except:
                real_price = 0

            # 인테리어 비용
            approval_elements = driver.find_elements(By.CSS_SELECTOR, "span.show_b")
            year = 9999
            for elem in approval_elements:
                text = elem.text.strip()
                if "사용승인일" in text:
                    year = int(text.split(":")[1].split("-")[0])
                    break

            if year < 2010:
                try:
                    size_text = driver.find_element(By.CSS_SELECTOR, "#Tr_4 > td:nth-child(3) > div:nth-child(6) > span").text
                    area_size = int(size_text.split("(")[1].split("평")[0])
                    interior = area_size * 1000000
                except:
                    interior = 3000000
            else:
                interior = 3000000
            
            # 주차대수
            try:
                parking = driver.find_element(By.CSS_SELECTOR, "#bldg_table > tr:nth-child(4) > td:nth-child(6)").text.split("대")[0]
                home = driver.find_element(By.CSS_SELECTOR, "#bldg_table > tr:nth-child(1) > td:nth-child(4)").text.split(" / ")[1]
                parking = home/parking
                print(f"✅ 주차대수 추출 성공: {parking}")
            except Exception as e:
                parking = None
                print(f"⚠️ 주차대수 추출 실패: {e}")
            
            # 보증금
            try:
                p = driver.find_element(By.CSS_SELECTOR, "#lyCnt_leas > table.Ltbl_list > tbody > tr.dtLeasTr > td:nth-child(6) > span").text
                if p == "있음" :
                    rent = int(driver.find_element(By.CSS_SELECTOR, "#lyCnt_leas > table.Ltbl_list > tbody > tr.dtLeasTr > td.right").text.replace(",","").split(":")[1].split("원")[0])
                else :
                    rent = 0
                print(f"✅ 보증금 추출 성공: {rent}")
            except Exception as e:
                parking = None
                print(f"⚠️ 보증금 추출 실패: {e}")
                
            expense = (start_price * 0.011) + (start_price * 0.04) + 2000000 + ((start_price - end_price) * 0.24) + interior + (end_price * 0.6 * 0.05 / 2) + 600000 + 1000000
            max_pice = end_price + expense + 20000000
            decision = 1 if max_pice < real_price and max_pice > rent else 0
            


            # 팝업 닫고 메인으로 복귀
            driver.close()
            driver.switch_to.window(main_window)

            # 결과 저장
            result1.append([decision, num, address, area, year,parking, start_price, end_price, max_pice, None, expense, real_price, average_sale_rate, None, hit_info, hit_info2, the_date])
            print(f"✅ {idx}번째 tr 추출 완료")

            time.sleep(2)

        except Exception as e:
            print(f"⚠️ {idx}번째 tr 추출 실패: {e}")

    # 다음 페이지 없으면 break
    if not move_to_next_page(driver):
        print("✅ 다음 페이지 없음, 크롤링 종료")
        break

# --- 결과 저장 ---
df = pd.DataFrame(result1, columns=cols)
df.to_csv(f"./[{today.year}년{today.month:02d}월]{keyword1}_{keyword2}_경매결과.csv", encoding="utf-8-sig", index=False)
print("✅ 파일 저장 완료")




🔍 리스트 로딩 완료
🔵 현재 페이지 tr 개수: 100개
✅ 예상 낙찰 비율 추출 성공: 56.58%
⚠️ 주차대수 추출 실패: unsupported operand type(s) for /: 'str' and 'str'
✅ 보증금 추출 성공: 255000000
✅ 1번째 tr 추출 완료
⚠️ 예상 낙찰 비율 추출 실패: 매각사례 테이블을 찾지 못했습니다.
⚠️ 주차대수 추출 실패: unsupported operand type(s) for /: 'str' and 'str'
✅ 보증금 추출 성공: 380000000
✅ 2번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 93.4%
⚠️ 주차대수 추출 실패: unsupported operand type(s) for /: 'str' and 'str'
✅ 보증금 추출 성공: 138600000
✅ 3번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 94.72%
⚠️ 주차대수 추출 실패: unsupported operand type(s) for /: 'str' and 'str'
⚠️ 보증금 추출 실패: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#lyCnt_leas > table.Ltbl_list > tbody > tr.dtLeasTr > td:nth-child(6) > span"}
  (Session info: chrome=135.0.7049.115); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0098D363+60275]
	GetHandleVerifier [0x0098D3A4+60340]
	(No symbol) [0x007

⚠️ 89번째 tr 추출 실패: Message: no such element: element not found
  (Session info: chrome=135.0.7049.115); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0098D363+60275]
	GetHandleVerifier [0x0098D3A4+60340]
	(No symbol) [0x007C06F3]
	(No symbol) [0x007D20E1]
	(No symbol) [0x007D11C0]
	(No symbol) [0x007C78F3]
	(No symbol) [0x007C5DFA]
	(No symbol) [0x007C913A]
	(No symbol) [0x007C91B7]
	(No symbol) [0x00807F24]
	(No symbol) [0x00808A2B]
	(No symbol) [0x007FDDD1]
	(No symbol) [0x0082D0D4]
	(No symbol) [0x007FDCF4]
	(No symbol) [0x0082D304]
	(No symbol) [0x0084E6EB]
	(No symbol) [0x0082CE86]
	(No symbol) [0x007FC623]
	(No symbol) [0x007FD474]
	GetHandleVerifier [0x00BD8FE3+2467827]
	GetHandleVerifier [0x00BD45E6+2448886]
	GetHandleVerifier [0x00BEF80C+2560028]
	GetHandleVerifier [0x009A3DF5+153093]
	GetHandleVerifier [0x009AA3BD+179149]
	GetHandleVerifier [0x00

* 주차대수 다시 찾아야함
* 물건번호가 여러개인경우 고려

In [ ]:
#bldg_table > tr:nth-child(4) > td:nth-child(6)

In [20]:
text = "보:225,000,000원"

In [25]:
int(text.replace(",","").split(":")[1].split("원")[0])

225000000